## Random Forest Classifier With Pipeline And Hyperparameter Tuning

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = sns.load_dataset("tips")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [4]:
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [5]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [6]:
df.time

0      Dinner
1      Dinner
2      Dinner
3      Dinner
4      Dinner
        ...  
239    Dinner
240    Dinner
241    Dinner
242    Dinner
243    Dinner
Name: time, Length: 244, dtype: category
Categories (2, object): ['Lunch', 'Dinner']

### Problem Statement
Here Time is our dependent variable, we have to predict the time i.e., Lunch or Dinner base on the available independent variables

In [7]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['time'] = encoder.fit_transform(df['time'])

In [8]:
df.time.unique()

array([0, 1])

In [9]:
# Independent and Dependent Feature
X = df.drop(labels=['time'], axis=1)
y = df['time']

In [10]:
y

0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int64

In [11]:
X['day'].value_counts()

Sat     87
Sun     76
Thur    62
Fri     19
Name: day, dtype: int64

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer # Handling Missing Values
from sklearn.preprocessing import StandardScaler # Feature Scaling
from sklearn.preprocessing import OneHotEncoder # Categorical to Numerical
from sklearn.compose import ColumnTransformer

In [14]:
# Separate Categorical and Numerical Columns
categorical_cols = ['sex', 'smoker','day']
numerical_cols = ['total_bill','tip','size']

In [15]:
## Feature Engineering Automation Using Pipeline

## Numerical Pipeline
num_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')),    # Missing Values
        ('scaler', StandardScaler())    # Feature Scaling
    ]
)

# Categorical Pipeline
cat_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),    # Missing Values
        ('onehotencoder', OneHotEncoder())    # Categorical Features to Numerical
    ]
)

In [16]:
preprocessor = ColumnTransformer([
    ('num_pipeline', num_pipeline, numerical_cols),
    ('cat_pipeline', cat_pipeline, categorical_cols)
])

preprocessor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['total_bill', 'tip', 'size']),
                                ('cat_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder())]),
                                 ['sex', 'smoker', 'day'])])

In [17]:
X_train.head()

,total_bill,tip,sex,smoker,day,size
115,17.31,3.50,Female,No,Sun,2
181,23.33,5.65,Male,Yes,Sun,2
225,16.27,2.50,Female,Yes,Fri,2
68,20.23,2.01,Male,No,Sat,2
104,20.92,4.08,Female,No,Sat,2


In [18]:
X_train = preprocessor.fit_transform(X_train)
X_train

array([[-0.33481585,  0.29210141, -0.61218506, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.34596381,  1.78563356, -0.61218506, ...,  0.        ,
         1.        ,  0.        ],
       [-0.45242562, -0.40256471, -0.61218506, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-1.64209372, -1.44456389, -0.61218506, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.62383924,  0.32683471, -0.61218506, ...,  0.        ,
         1.        ,  0.        ],
       [ 2.71738397, -0.40256471,  0.45476605, ...,  1.        ,
         0.        ,  0.        ]])

In [19]:
X_test.head()

,total_bill,tip,sex,smoker,day,size
24,19.82,3.18,Male,No,Sat,2
6,8.77,2.00,Male,No,Sun,2
153,24.55,2.00,Male,No,Sun,4
211,25.89,5.16,Male,Yes,Sat,4
198,13.00,2.00,Female,Yes,Thur,2


In [20]:
X_test = preprocessor.transform(X_test)
X_test

array([[-5.09691792e-02,  6.98082509e-02, -6.12185061e-01,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [-1.30057303e+00, -7.49897768e-01, -6.12185061e-01,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00],
       [ 4.83929121e-01, -7.49897768e-01,  1.52171715e+00,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00],
       [ 6.35464792e-01,  1.44524716e+00,  1.52171715e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [-8.22217891e-01, -7.49897768e-01, -6.12185061e-01,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
  

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [22]:
# Model Training Automation
models={
    'Logistic Regression' : LogisticRegression(),
    'Decision Tree' : DecisionTreeClassifier(),
    'Random Forest' : RandomForestClassifier()
}

In [23]:
from sklearn.metrics import accuracy_score

In [24]:
def evaluate_model(X_train, X_test, y_train, y_test, models):
    report={}
    for i in range(len(models)):
        model = list(models.values())[i]
        
        # Train Model
        model.fit(X_train, y_train)
        
        # Predict Testing data
        y_test_pred = model.predict(X_test)
        
        # Get Accuracy for test data prediction
        test_model_score = accuracy_score(y_test, y_test_pred)
        
        report[list(models.keys())[i]] = test_model_score
        
    return report

In [25]:
evaluate_model(X_train, X_test, y_train, y_test, models)

{'Logistic Regression': 1.0,
 'Decision Tree': 0.9508196721311475,
 'Random Forest': 0.9672131147540983}

### Hyperparameter Tuning

In [26]:
classifier = RandomForestClassifier()

In [27]:
parameters = {
    'max_depth' : [3,5,10,None],
    'n_estimators' : [100, 200, 300],
    'criterion' : ['gini', 'entropy']
}

In [28]:
from sklearn.model_selection import RandomizedSearchCV
cv = RandomizedSearchCV(classifier, param_distributions=parameters, scoring = 'accuracy', cv = 5, verbose=3)


In [29]:
cv

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [30]:
cv.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=entropy, max_depth=10, n_estimators=100;, score=0.946 total time=   0.2s
[CV 2/5] END criterion=entropy, max_depth=10, n_estimators=100;, score=0.973 total time=   0.2s
[CV 3/5] END criterion=entropy, max_depth=10, n_estimators=100;, score=1.000 total time=   0.2s
[CV 4/5] END criterion=entropy, max_depth=10, n_estimators=100;, score=0.944 total time=   0.2s
[CV 5/5] END criterion=entropy, max_depth=10, n_estimators=100;, score=0.917 total time=   0.2s
[CV 1/5] END criterion=entropy, max_depth=3, n_estimators=300;, score=0.946 total time=   0.6s
[CV 2/5] END criterion=entropy, max_depth=3, n_estimators=300;, score=0.973 total time=   0.5s
[CV 3/5] END criterion=entropy, max_depth=3, n_estimators=300;, score=0.973 total time=   0.5s
[CV 4/5] END criterion=entropy, max_depth=3, n_estimators=300;, score=0.917 total time=   0.5s
[CV 5/5] END criterion=entropy, max_depth=3, n_estimators=300;, score=0.917 tot

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [31]:
cv.best_params_

{'n_estimators': 300, 'max_depth': 5, 'criterion': 'entropy'}

In [32]:
# Random Forest Regression

In [33]:
df = sns.load_dataset('tips')

In [34]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [36]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64